#GW 16384 (4s)

In [ ]:
pip install gwpy lalsuite Pycbc

In [ ]:
import numpy as np
from pycbc.waveform import get_td_waveform
from gwpy.timeseries import TimeSeries
from tqdm import tqdm
#-------------------------
#       Deep learning
#import torch.nn as nn 
#import torch 
#import torchvision.datasets as dsets
#from torch.utils.data import Dataset, DataLoader
#import torch.optim as optim
#---Data reading and writing---------------
import csv
import h5py
import pandas as pd
from scipy import signal
import scipy.io.wavfile as s
import matplotlib.pyplot as plt


In [ ]:
signal_gw=np.zeros((67800,16384))
next_val=0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

In [ ]:
ls gdrive/My\ Drive/GW\ data/

In [ ]:
"""a=['5','10','15','20','5','10','15','20','50','100','150','200','250','300','350','400','450','500','.5','.8','.5','.5','.8','.5','60','80','120','50','UIE','CIE','250','260','270','280','Glitch','I','II','III','10','20','30','30','60','Noise']
print(a[43])"""

In [ ]:
next_val=0
#------------------------------------------------------------------
#                    TRAINING DATASETS PREPRATION               ---
#------------------------------------------------------------------

class dataprep_train:

#------------------------------------------------------------------
#                    SIMULATED SIGNALS+TRANSIENT NOISE
#------------------------------------------------------------------
  def simulated_signals(noise):  
    global next_val,signal_gw
    apx=['TaylorT1', 'TaylorT2', 'SpinTaylorT4','SpinTaylorT5','EOBNRv2' , 'SEOBNRv1', 'SEOBNRv2','IMRPhenomB']
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for a in tqdm(range(len(apx))):
        check=np.zeros(noise.shape[1])
        k=0
        spin1x,spin1y,spin1z,spin2x,spin2y,spin2z=0,0,0,0,0,0
        if apx=='TaylorT1':
          for m1 in range(5,21,5):
            for m2 in range(5,21,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)

        elif apx=='TaylorT2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    
        elif apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [60,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      spin1x=0.5,spin1y=0.8,spin1z=0.5,
                                      spin2x=0.5,spin2y=0.8,spin2z=0.5,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
          
        elif apx=='EOBNRv2' or apx=='SEOBNRv1' or apx=='SEOBNRv2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
        else:
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    if len(hp)<=16385:
                      signal_gw[next_val:next_val+noise.shape[0],:]=np.copy(noise)
                      pos=np.random.randint(0,noise.shape[1]-len(hp))
                      signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(hp)]+=hp
                      writer = csv.writer(file)
                      for i in range(noise.shape[0]):
                        col=np.zeros(44)
                        if apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
                          col[m1//5],col[3+m2//4],col[7+d//50],col[18],col[19],col[20],col[21],col[22],col[23],col[23+fl//40],col[27]=1,1,1,1,1,1,1,1,1,1,1
                        else:
                          col[m1//5],col[3+m2//4],col[7+d//50],col[23+fl//40],col[27]=1,1,1,1,1
                        col[43]=1
                        writer.writerow(col)
                      k=k+1
                      next_val+=(noise.shape[0])
                    
#------------------------------------------------------------------
#                    ECHOES +TRANSIENT NOISE
#------------------------------------------------------------------
  def echoes(noise):
    global next_val,signal_gw
    signal_gw[next_val:next_val+noise.shape[0],:]=np.copy(noise)
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for loop in tqdm(range(10)):##############################10
        t=np.linspace(0,.3,np.random.randint(16384))
        y1,y2=np.zeros(len(t)),np.zeros(len(t))
        i=0
        for j in range(8):
          for f in [250,260,270,280]:
            for i in range(len(t)):
              aa=t[i]-0.0295-j*0.0295
              y1[i]+=1.5*10e-21*(-1)**j*(1.5*10e-21*.5/(3+j))*np.exp(-(aa**2)/(2*.006**2))*np.cos(2*np.pi*f*aa)
            pos=np.random.randint(0,noise.shape[1]-len(t))
            signal_gw[next_val:next_val+noise.shape[0]-1,pos:pos+len(t)]+=y1
            next_val+=(noise.shape[0])
            writer = csv.writer(file)
            for i in range(noise.shape[0]):
              col=np.zeros(44)
              col[28],col[29+(f//10)-24],col[43]=1,1,1
              writer.writerow(col)

        r=.3
        for j in range(8):
          for f in [250,260,270,280]:
            for i in range(len(t)):
              aa=t[i]-0.0295-j*0.0295-(j*(j+1)/2)*r*0.0295
              y2[i]+=1.5*10e-21*(-1)**j*(1.5*10e-21*.5/(3+j))*np.exp(-(aa**2)/(2*.006**2))*np.cos(2*np.pi*f*aa)
            pos=np.random.randint(0,noise.shape[1]-len(t))
            signal_gw[next_val:next_val+noise.shape[0]-1,pos:pos+len(t)]+=y2
            next_val+=(noise.shape[0])
            writer = csv.writer(file)          
            for i in range(noise.shape[0]):
                  col=np.zeros(44)
                  col[29],col[29+(f//10)-24],col[43]=1,1,1
                  writer.writerow(col)

#------------------------------------------------------------------
#                    GLITCHES+TRANSIENT NOISE
#------------------------------------------------------------------
  def glitches(noise):
    global next_val,signal_gw
    signal_gw[next_val:next_val+noise.shape[0],:]=np.copy(noise)
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for i in tqdm(['helix2','whistle','wandering_line','violin_mode','Tomte','koi_fish','scratchy','scattered_light','repeating_blip','power_line','paired_dove','low_freq_burst','light_modulation']):
        loc='gdrive/My Drive/GW data/Glitches/'+i+'.wav'
        rate,data=s.read(loc)
        for c1 in range(5,9):
          for c2 in range(3,13,2):
            b, a = signal.butter(c1, .9, btype='lowpass', analog=False)
            low_passed = signal.filtfilt(b, a, data)
            y = 10e-28*signal.medfilt(low_passed,c2)
            k=np.zeros(8820)
            j=0
            for i in range(0,len(y),5):
              k[j]=y[i]
              j+=1
            pos=np.random.randint(0,noise.shape[1]-len(k))
            signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(k)]+=k
            next_val+=(noise.shape[0])
            writer = csv.writer(file)
            for i in range(noise.shape[0]):
              col=np.zeros(44)
              col[34],col[43]=1,1
              writer.writerow(col)

#------------------------------------------------------------------------------
#                             CCSNe+Transient Noise               -------------
#------------------------------------------------------------------------------
  def ccsne(noise):
    global next_val,signal_gw
    signal_gw[next_val:next_val+noise.shape[0]]=np.copy(noise)
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      val=['signal_A1B1G1_R.dat','signal_A1B2G1_R.dat','signal_A1B3G1_R.dat','signal_A1B3G2_R.dat','signal_A1B3G3_R.dat','signal_A1B3G5_R.dat','signal_A2B4G1_R.dat','signal_A3B1G1_R.dat','signal_A3B2G1_R.dat','signal_A3B2G2_R.dat','signal_A3B2G4_soft_R.dat','signal_A3B2G4_R.dat','signal_A3B3G1_R.dat','signal_A3B3G2_R.dat','signal_A3B3G3_R.dat','signal_A3B3G5_R.dat','signal_A3B4G2_R.dat','signal_A3B5G4_R.dat','signal_A4B1G1_R.dat','signal_A4B1G2_R.dat','signal_A4B2G2_R.dat','signal_A4B2G3_R.dat','signal_A4B4G4_R.dat','signal_A4B4G5_R.dat','signal_A4B5G4_R.dat','signal_A4B5G5_R.dat']
      for aak in tqdm(val):
        loc='gdrive/My Drive/GW data/CCSNe/'+aak
        x, y = np.loadtxt(loc,unpack=True, usecols=[0,1])
        for r in [10,20,30]:
          for theta in [30,60]:
            y = 1/8*np.sqrt(15/np.pi)*y/r*(np.sin(theta))**2
            new_arr=np.zeros(16000)
            j=0
            for i in range(0,len(y),5):
              new_arr[j]=y[i]
              j+=1
            pos=np.random.randint(0,noise.shape[1]-len(new_arr))
            signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(new_arr)]+=new_arr
            writer = csv.writer(file)
            for i in range(noise.shape[0]):
              col=np.zeros(44)
              if val=='signal_A2B4G1_R.dat'  or   val=='signal_A3B3G5_R.dat' or  val=='signal_A3B5G4_R.dat' :
                col[37]=1
              elif val=='signal_A3B3G1_R.dat'  or val=='signal_A3B4G2_R.dat':
                col[36]=1
              elif val=='signal_A3B3G2_R.dat' or  val=='signal_A3B3G3_R.dat' or  val=='signal_A4B2G2_R.dat' or  val=='signal_A4B2G3_R.dat' or  val=='signal_A4B4G4_R.dat' or  val=='signal_A4B4G5_R.dat' or  val=='signal_A4B5G4_R.dat' or  val=='signal_A4B5G5_R.dat':
                col[36],col[35]=1,1
              else:
                col[35]=1
              col[37+r//10],col[40+theta//30],col[43]=1,1,1
              writer.writerow(col)
            next_val=next_val+noise.shape[0]    

#------------------------------------------------------------------------------
#                             MIXED SIGNALS                       -------------
#------------------------------------------------------------------------------
  def mixed_signals_BHBNSB(noise):
    global next_val,signal_gw
    apx=['TaylorT1', 'TaylorT2', 'SpinTaylorT4', 'SpinTaylorT5','EOBNRv2' , 'SEOBNRv1', 'SEOBNRv2','IMRPhenomB']
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for a in tqdm(range(len(apx))):
        check=np.zeros(noise.shape[1])
        k=0
        spin1x,spin1y,spin1z,spin2x,spin2y,spin2z=0,0,0,0,0,0
        if apx=='TaylorT1':
          for m1 in range(5,21,5):
            for m2 in range(5,21,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)

        elif apx=='TaylorT2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    
        elif apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [60,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      spin1x=0.5,spin1y=0.8,spin1z=0.5,
                                      spin2x=0.5,spin2y=0.8,spin2z=0.5,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
          
        elif apx=='EOBNRv2' or apx=='SEOBNRv1' or apx=='SEOBNRv2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
        else:
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    if len(hp)<=16385:
                      for loops in range(2):####10
                        t=np.linspace(0,.3,np.random.randint(16384))
                        y2=np.zeros(len(t))          
                        r=.3
                        for j in range(3,8):
                          for i in range(len(t)):
                            aa=t[i]-0.0295-j*0.0295-(j*(j+1)/2)*r*0.0295
                            y2[i]+=1.5*10e-21*(-1)**j*(1.5*10e-21*.5/(3+j))*np.exp(-(aa**2)/(2*.006**2))*np.cos(2*np.pi*250*aa)

                          #wave=['helix2']#,'whistle','wandering_line','violin_mode','Tomte']:#,'koi_fish','scratchy','scattered_light','repeating_blip','power_line','paired_dove','low_freq_burst','light_modulation']:
                          #for wave_name in wave: 
                          #  loc='gdrive/My Drive/GW data/Glitches/'+wave_name+'.wav'
                          #  rate,data=s.read(loc)
                          #  for c2 in [5,7]:
                          #    b, a = signal.butter(7, .9, btype='lowpass', analog=False)
                          #    low_passed = signal.filtfilt(b, a, data)
                          #    z = 10e-28*signal.medfilt(low_passed,c2)
                          #    sig=np.zeros(8820)
                          #    jj=0
                          #    for ii in range(0,len(z),5):
                          #      sig[jj]=z[ii]
                          #      jj+=1

                          signal_gw[next_val:next_val+noise.shape[0]]=np.copy(noise)
                          pos=np.random.randint(0,noise.shape[1]-len(hp))
                          signal_gw[next_val:next_val+noise.shape[0] ,pos:pos+len(hp)]+=hp
                          #pos=np.random.randint(0,noise.shape[1]-len(sig))
                          #signal_gw[next_val:next_val+noise.shape[0] ,pos:pos+len(sig)]+=sig
                          pos=np.random.randint(0,noise.shape[1]-len(y2))                                  
                          signal_gw[next_val:next_val+noise.shape[0] ,pos:pos+len(y2)]+=y2
                          writer = csv.writer(file)
                          for i in range(noise.shape[0]):
                            col=np.zeros(44)
                            if apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
                              col[m1//5],col[3+m2//4],col[7+d//50],col[18],col[19],col[20],col[21],col[22],col[23],col[23+fl//40],col[27]=1,1,1,1,1,1,1,1,1,1,1
                            else:
                              col[m1//5],col[3+m2//4],col[7+d//50],col[23+fl//40],col[27]=1,1,1,1,1
                            col[29],col[29+(25//10)-24]=1,1
                            writer.writerow(col)  
                            k=k+1
                          next_val+=(noise.shape[0])
                                #for i in range(noise.shape[0]):
                                #  col=np.zeros(44)
                                #  if apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
                                #    col[m1//5],col[3+m2//4],col[7+d//50],col[18],col[19],col[20],col[21],col[22],col[23],col[23+fl//40],col[27]=1,1,1,1,1,1,1,1,1,1,1
                                #  else:
                                #    col[m1//5],col[3+m2//4],col[7+d//50],col[23+fl//40],col[27]=1,1,1,1,1
                                #  col[29],col[29+(25//10)-24],col[34]=1,1,1
                                #writer.writerow(col)                         




#------------------------------------------------------------------------------
#                             MIXED SIGNALS                       -------------
#------------------------------------------------------------------------------
  def mixed_signals_CCSNe(noise):
    global next_val,signal_gw  
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      val=['signal_A1B1G1_R.dat','signal_A1B2G1_R.dat','signal_A1B3G1_R.dat','signal_A1B3G2_R.dat','signal_A1B3G3_R.dat','signal_A1B3G5_R.dat','signal_A2B4G1_R.dat','signal_A3B1G1_R.dat','signal_A3B2G1_R.dat','signal_A3B2G2_R.dat','signal_A3B2G4_soft_R.dat','signal_A3B2G4_R.dat','signal_A3B3G1_R.dat','signal_A3B3G2_R.dat','signal_A3B3G3_R.dat','signal_A3B3G5_R.dat','signal_A3B4G2_R.dat','signal_A3B5G4_R.dat','signal_A4B1G1_R.dat','signal_A4B1G2_R.dat','signal_A4B2G2_R.dat','signal_A4B2G3_R.dat','signal_A4B4G4_R.dat','signal_A4B4G5_R.dat','signal_A4B5G4_R.dat','signal_A4B5G5_R.dat']
      for aak in val:
        loc='gdrive/My Drive/GW data/CCSNe/'+aak
        x, y = np.loadtxt(loc,unpack=True, usecols=[0,1])
        for r in [10,20,30]:
          for theta in [30,60]:
            y = 1/8*np.sqrt(15/np.pi)*y/r*(np.sin(theta))**2
            new_arr=np.zeros(16000)
            j=0
            for i in range(0,len(y),5):
              new_arr[j]=y[i]
              j+=1
            wave=['helix2','whistle','wandering_line','violin_mode','Tomte','koi_fish','scratchy','scattered_light','repeating_blip','power_line','paired_dove','low_freq_burst','light_modulation']
            for wave_name in wave: 
              loc='gdrive/My Drive/GW data/Glitches/'+wave_name+'.wav'
              rate,data=s.read(loc)
              for c2 in [5,7]:
                b, a = signal.butter(7, .9, btype='lowpass', analog=False)
                low_passed = signal.filtfilt(b, a, data)
                z = 10e-28*signal.medfilt(low_passed,c2)
                sig=np.zeros(8820)
                jj=0
                for ii in range(0,len(z),5):
                  sig[jj]=z[ii]
                  jj+=1
                signal_gw[next_val:next_val+noise.shape[0]]=np.copy(noise)
                pos=np.random.randint(0,noise.shape[1]-len(sig)) 
                signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(sig)]+=sig 
                pos=np.random.randint(0,noise.shape[1]-len(new_arr))
                signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(new_arr)]+=new_arr                              
                writer = csv.writer(file)
                for i in range(noise.shape[0]):
                  col=np.zeros(44)
                  if val=='signal_A2B4G1_R.dat' or  val=='signal_A3B3G5_R.dat' or  val=='signal_A3B5G4_R.dat' :
                    col[37]=1
                  elif val=='signal_A3B3G1_R.dat' or val=='signal_A3B4G2_R.dat':
                    col[36]=1
                  elif val=='signal_A3B3G2_R.dat' or  val=='signal_A3B3G3_R.dat' or  val=='signal_A4B2G2_R.dat' or  val=='signal_A4B2G3_R.dat' or  val=='signal_A4B4G4_R.dat' or  val=='signal_A4B4G5_R.dat' or  val=='signal_A4B5G4_R.dat' or  val=='signal_A4B5G5_R.dat':
                    col[36],col[35]=1,1
                  else:
                    col[35]=1
                  col[34],col[37+r//10],col[40+theta//30],col[43]=1,1,1,1
                  writer.writerow(col)
                next_val+=noise.shape[0]



#----------------------------------------------------------------------------------
#                               PIPELINES                                       ---
#----------------------------------------------------------------------------------
def train_pipeline(noise):
  global next_val,signal_gw
  val=dataprep_train
  with open('gdrive/My Drive/GW data/labels_mixed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['5','10','15','20','5','10','15','20','50','100','150','200','250','300','350','400','450','500','.5','.8','.5','.5','.8','.5','60','80','120','50','UIE','CIE','250','260','270','280','Glitch','I','II','III','10','20','30','30','60','Noise'])
  print('\nSimulated GW .......')    
  val.simulated_signals(noise)
  print('\nSimulated GW training set 100%')
  print('data size :'+ str(next_val)+'\n')  
  print('\nEchoes...')
  val.echoes(noise)
  print('data size :'+ str(next_val)+'\n')
  print('\nEchoes 100%')
  print('\nCCSNE...')  
  val.ccsne(noise)
  print('data size :'+ str(next_val)+'\n')
  print('\nCCSNE 100%')
  print('\nGlitches...')    
  val.glitches(noise)
  print('data size :'+ str(next_val)+'\n')  
  print('\nGlitches 100%')  
  print('\nMixed training BHBNSB ...' )
  val.mixed_signals_BHBNSB(noise)
  print('\nMixed set BHBNSB 100%')
  print('data size :'+ str(next_val)+'\n')
  print('\nMixed training CCSNe...' )
  val.mixed_signals_CCSNe(noise)
  print('\nMixed set CCSNe 100%')
  print('data size :'+ str(next_val)+'\n')  
  #hf = h5py.File('gdrive/My Drive/GW data/mixed.h5', 'w')
  #hf.create_dataset('mixed', data=signal_gw)
  #hf.close()

In [ ]:
#--------------------------------------------------------
#                           MAIN                    -----
#--------------------------------------------------------
#if __name__ == '__main__':
  #global next_val
  #print('\nCollecting noise.....\n data size : '+str(next_val))
hf= h5py.File('gdrive/My Drive/GW data/noise.h5', 'r')
group_key = list(hf.keys())[0]
noise= hf[group_key]
noise=np.array(noise)
print(noise.shape,type(noise))
hf.close()
noise=noise[:5]
print(noise.shape)

In [ ]:
val=np.sum(noise,axis=1)
#val=val-min(val)
plt.hist(val)

In [ ]:
#noise=np.zeros((50,16384))
#for i in tqdm(range(50)):
#  noise[i]=TimeSeries.fetch_open_data('H1', 1164726272+4*i, 1164726276+4*i)

In [ ]:
print('\n\nPreparing data..... ')
train_pipeline(noise) #--------training-------------#
print('\nPreparing data........100%\n\n')

In [ ]:
labels=pd.read_csv('gdrive/My Drive/GW data/labels_mixed.csv')
labels.shape

In [ ]:
print(next_val)

In [ ]:
labels.columns.shape

In [ ]:
val.shape

In [ ]:
val

In [ ]:
val=np.sum(labels,axis=0)
plt.hist(val)

In [ ]:
val=np.sum(signal_gw,axis=1)
#plt.figure(figsize=(20,10))
plt.hist(val)

# Movies Genre

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [ ]:
train = pd.read_csv('gdrive/My Drive/train.csv')    # reading the csv file
train.head()      # printing first five rows of the file

In [ ]:
train.columns

In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('gdrive/My Drive/Images/'+train['Id'][i]+'.jpg',target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

In [ ]:
X.shape

In [ ]:
plt.imshow(X[2])

In [ ]:
train['Genre'][2]

In [ ]:
y = np.array(train.drop(['Id', 'Genre'],axis=1))
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
img = image.load_img('GOT.jpg',target_size=(400,400,3))
img = image.img_to_array(img)
img = img/255

In [ ]:
classes = np.array(train.columns[2:])
proba = model.predict(img.reshape(1,400,400,3))
top_3 = np.argsort(proba[0])[:-4:-1]
for i in range(3):
    print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
plt.imshow(img)

In [ ]:
img = image.load_img('avengers.jpeg',target_size=(400,400,3))
img = image.img_to_array(img)
img = img/255

In [ ]:
classes = np.array(train.columns[2:])
proba = model.predict(img.reshape(1,400,400,3))
top_3 = np.argsort(proba[0])[:-4:-1]
for i in range(3):
    print("{}".format(classes[top_3[i]])+" ({:.3})".format(proba[0][top_3[i]]))
plt.imshow(img)

#Classification using Keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [ ]:
data=signal_gw[:next_val]
print(data.shape)
data=data.reshape((next_val,data.shape[1],1))
print(data.shape)

In [ ]:
labels=pd.read_csv('gdrive/My Drive/GW data/labels_mixed.csv')
labels.head

In [ ]:
labels=labels.to_numpy(dtype=int,copy=True)

In [ ]:
labels.shape

In [ ]:
graph=np.sum(labels,axis=1)

In [ ]:
plt.hist(graph)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=1, test_size=0.2)

In [ ]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

In [ ]:
model = Sequential()
model.add(Conv1D(filters=4,   kernel_size=4, activation="relu", input_shape=(X_train.shape[1],1)))#1
model.add(Dropout(0.2))
model.add(Conv1D(filters=8,   kernel_size=4, activation="relu", input_shape=(X_train.shape[1],1)))#1
model.add(Dropout(0.2))
model.add(Conv1D(filters=16,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#2
model.add(Dropout(0.2))
model.add(Conv1D(filters=32,  kernel_size=4, activation='relu',padding='valid',use_bias='True'))#3
model.add(Dropout(0.3))
model.add(Conv1D(filters=64,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#4
model.add(Dropout(0.3))
model.add(Conv1D(filters=128, kernel_size=4, activation="relu",padding='valid',use_bias='True'))#5
model.add(Dropout(0.3))
model.add(Conv1D(filters=256, kernel_size=4, activation="relu",padding='valid',use_bias='True'))#6
model.add(Dropout(0.3))
#model.add(Conv1D(filters=256, kernel_size=5, activation="relu",padding='valid',use_bias='True'))#7
#model.add(Conv1D(filters=256, kernel_size=5, activation="relu",padding='valid',use_bias='True'))#8
model.add(Conv1D(filters=128, kernel_size=4, activation="relu",padding='valid',use_bias='True'))#12
model.add(Dropout(0.2))
model.add(Conv1D(filters=64,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#13
model.add(Dropout(0.2))
model.add(Conv1D(filters=32,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#14
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(44, activation='sigmoid'))

In [ ]:
sgd = SGD(lr=1e-6,decay=1e-6, momentum=0.99,nesterov=True, clipnorm=1.0)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
a=model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=500)

In [ ]:
plt.plot(a.history['accuracy'])
plt.plot(a.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

#DL multi classification Pytorch

In [ ]:
#------------------------------------------------------------------
#                   DEEP LEARNING MODEL                     -------
#------------------------------------------------------------------
class PrepData(Dataset):
  def __init__(self,images,labels):
    self.image=images
    self.label=labels
  def __len__(self):
    return len(self.image)
  def __getitem__(self,index):
    data = self.image[index]
    data=data.unsqueeze(0)
    return (data,self.label[index])


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.cnn1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        self.cnn2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()        
        self.cnn3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu3 = nn.ReLU()  
        self.cnn4 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=2)
        self.relu4 = nn.ReLU()  
        self.cnn5 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.relu5 = nn.ReLU()  

        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.cnn6 = nn.Conv1d(in_channels=64, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.relu6 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(in_features=131072, out_features=43,bias=True) 

    def forward(self, x):
        out = self.relu1(self.cnn1(x))
        out = self.relu2(self.cnn2(out))
        out = self.relu3(self.cnn3(out))
        out = self.relu4(self.cnn4(out))
            
        out = self.maxpool1(self.relu5(self.cnn5(out)))
        out = self.maxpool2(self.relu6(self.cnn6(out)))
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out

def training(train_loader,test_loader,optimizer,criterion,neuron):
 iter = 0
 for epoch in (range(2)):
    for i, (images, labels) in enumerate(train_loader):
        images = images.requires_grad_()
        optimizer.zero_grad()

        outputs = neuron(images.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iter += 1

    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.requires_grad_()
        outputs = neuron(images.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()
        accuracy = 100 * correct / total
    print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

def model_prep(train,test,train_labels,test_labels):
  neuron=CNN()
  #print(neuron)
  criterion=nn.BCELoss()
  optimizer=optim.SGD(neuron.parameters(),1e-2, momentum = 0.9,weight_decay = 1e-5)
  train_data=PrepData(train,train_labels)
  test_data=PrepData(test,test_labels)
  train_loader = DataLoader(dataset=train_data,batch_size=100,shuffle=True)
  test_loader=DataLoader(dataset=test_data,batch_size=100,shuffle=False)
  a,b=next(iter(test_loader))
  print(a.shape,b.shape)
  training(train_loader,test_loader,optimizer,criterion,neuron)

def dl_model():
  global next_val
  #-------------------------------------labels---------------------------------
  labels=pd.read_csv('gdrive/My Drive/GW data/labels.csv')
  #label_id_only=pd.read_csv('gdrive/My Drive/GW data/labels.csv')
  #arr = label_id_only.to_numpy(dtype=int,copy=True)
  #print(labels.head)
  #label_id=np.zeros(12024)
  #for i in range(len(arr)):
  #  label_id[i]=arr[i][0]
  label_id=torch.from_numpy(labels.to_numpy(dtype=int,copy=True))
  print(labels.shape)
  #-------------------------------------loading data---------------------------------
  #hf_train = h5py.File('gdrive/My Drive/GW data/dataset.h5', 'r')
  #group_key = list(hf_train.keys())[0]
  #data= hf_train[group_key]
  data=np.copy(signal_gw)
  data = torch.from_numpy(np.array(data))
  print(data.shape)
  data = torch.from_numpy(np.array(signal_gw[:15360,:]))
  print(data.shape,label_id.shape)
  train,test=data[:15360-2000,:],data[15360-2000+1:,:]
  train_labels,test_labels=label_id[:15360-2000],label_id[15360-2000+1:]
  print("train:",train.shape,train_labels.shape,'\n',"test:",test.shape,test_labels.shape)
  model_prep(train,test,train_labels,test_labels)

#GW 8192 (2s)

In [ ]:
pip install gwpy lalsuite Pycbc

     |████████████████████████████████| 1.4MB 5.0MB/s 
     |████████████████████████████████| 29.0MB 109kB/s 
     |████████████████████████████████| 3.2MB 46.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 1.6MB 37.2MB/s 
     |████████████████████████████████| 798kB 35.0MB/s 
     |████████████████████████████████| 81kB 10.1MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 2.7MB 41.9MB/s 
  Created wheel for Pycbc: filename=PyCBC-1.16.1-cp36-cp36m-linux_x86_64.whl size=5636948 sha256=207923520826273e536f8da269777ad19a73a15efefa650b50cb11437cd85096
  Stored in directory: /root/.cache/pip/wheels/73/03/a1/e933f9ab69b7baf08f483e2fc15004fafe099c2e47c3c7f205
Successfully built Pycbc
  Created wheel for ligo-segments: filename=ligo_segments-1.2.0-cp36-cp36m-linux_x

In [ ]:
import numpy as np
from pycbc.waveform import get_td_waveform
from gwpy.timeseries import TimeSeries
from tqdm import tqdm
#-------------------------
#       Deep learning
#import torch.nn as nn 
#import torch 
#import torchvision.datasets as dsets
#from torch.utils.data import Dataset, DataLoader
#import torch.optim as optim
#---Data reading and writing---------------
import csv
import h5py
import pandas as pd
from scipy import signal
import scipy.io.wavfile as s
import matplotlib.pyplot as plt


In [ ]:
signal_gw=np.zeros((67800,8192))
next_val=0

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
ls gdrive/My\ Drive/GW\ data/

 1164685312_4096.hdf5   CCSNe_mixed.h5    labels.csv         noise.h5
 CCSNe/                 Glitches/         labels_final.csv
 CCSNe_mixed.csv       'Glitch images'/   labels_mixed.csv


In [ ]:
"""a=['5','10','15','20','5','10','15','20','50','100','150','200','250','300','350','400','450','500','.5','.8','.5','.5','.8','.5','60','80','120','50','UIE','CIE','250','260','270','280','Glitch','I','II','III','10','20','30','30','60','Noise']
print(a[43])"""

"a=['5','10','15','20','5','10','15','20','50','100','150','200','250','300','350','400','450','500','.5','.8','.5','.5','.8','.5','60','80','120','50','UIE','CIE','250','260','270','280','Glitch','I','II','III','10','20','30','30','60','Noise']\nprint(a[43])"

In [ ]:
next_val=0
#------------------------------------------------------------------
#                    TRAINING DATASETS PREPRATION               ---
#------------------------------------------------------------------

class dataprep_train:

#------------------------------------------------------------------
#                    SIMULATED SIGNALS+TRANSIENT NOISE
#------------------------------------------------------------------
  def simulated_signals(noise):  
    global next_val,signal_gw
    apx=['TaylorT1', 'TaylorT2', 'SpinTaylorT4','SpinTaylorT5','EOBNRv2' , 'SEOBNRv1', 'SEOBNRv2','IMRPhenomB']
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for a in tqdm(range(len(apx))):
        check=np.zeros(noise.shape[1])
        k=0
        spin1x,spin1y,spin1z,spin2x,spin2y,spin2z=0,0,0,0,0,0
        if apx=='TaylorT1':
          for m1 in range(5,21,5):
            for m2 in range(5,21,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)

        elif apx=='TaylorT2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    
        elif apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [60,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      spin1x=0.5,spin1y=0.8,spin1z=0.5,
                                      spin2x=0.5,spin2y=0.8,spin2z=0.5,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
          
        elif apx=='EOBNRv2' or apx=='SEOBNRv1' or apx=='SEOBNRv2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
        else:
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    if len(hp)<=8192:
                      signal_gw[next_val:next_val+noise.shape[0],:]=np.copy(noise)
                      pos=np.random.randint(0,noise.shape[1]-len(hp))
                      signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(hp)]+=hp
                      writer = csv.writer(file)
                      for i in range(noise.shape[0]):
                        col=np.zeros(44)
                        if apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
                          col[m1//5],col[3+m2//4],col[7+d//50],col[18],col[19],col[20],col[21],col[22],col[23],col[23+fl//40],col[27]=1,1,1,1,1,1,1,1,1,1,1
                        else:
                          col[m1//5],col[3+m2//4],col[7+d//50],col[23+fl//40],col[27]=1,1,1,1,1
                        col[43]=1
                        writer.writerow(col)
                      k=k+1
                      next_val+=(noise.shape[0])
                    
#------------------------------------------------------------------
#                    ECHOES +TRANSIENT NOISE
#------------------------------------------------------------------
  def echoes(noise):
    global next_val,signal_gw
    signal_gw[next_val:next_val+noise.shape[0],:]=np.copy(noise)
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for loop in tqdm(range(10)):##############################10
        t=np.linspace(0,.3,np.random.randint(8192))
        y1,y2=np.zeros(len(t)),np.zeros(len(t))
        i=0
        for j in range(8):
          for f in [250,260,270,280]:
            for i in range(len(t)):
              aa=t[i]-0.0295-j*0.0295
              y1[i]+=1.5*10e-21*(-1)**j*(1.5*10e-21*.5/(3+j))*np.exp(-(aa**2)/(2*.006**2))*np.cos(2*np.pi*f*aa)
            pos=np.random.randint(0,noise.shape[1]-len(t))
            signal_gw[next_val:next_val+noise.shape[0]-1,pos:pos+len(t)]+=y1
            next_val+=(noise.shape[0])
            writer = csv.writer(file)
            for i in range(noise.shape[0]):
              col=np.zeros(44)
              col[28],col[29+(f//10)-24],col[43]=1,1,1
              writer.writerow(col)

        r=.3
        for j in range(8):
          for f in [250,260,270,280]:
            for i in range(len(t)):
              aa=t[i]-0.0295-j*0.0295-(j*(j+1)/2)*r*0.0295
              y2[i]+=1.5*10e-21*(-1)**j*(1.5*10e-21*.5/(3+j))*np.exp(-(aa**2)/(2*.006**2))*np.cos(2*np.pi*f*aa)
            pos=np.random.randint(0,noise.shape[1]-len(t))
            signal_gw[next_val:next_val+noise.shape[0]-1,pos:pos+len(t)]+=y2
            next_val+=(noise.shape[0])
            writer = csv.writer(file)          
            for i in range(noise.shape[0]):
                  col=np.zeros(44)
                  col[29],col[29+(f//10)-24],col[43]=1,1,1
                  writer.writerow(col)

#------------------------------------------------------------------
#                    GLITCHES+TRANSIENT NOISE
#------------------------------------------------------------------
  def glitches(noise):
    global next_val,signal_gw
    signal_gw[next_val:next_val+noise.shape[0],:]=np.copy(noise)
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for i in tqdm(['helix2','whistle','wandering_line','violin_mode','Tomte','koi_fish','scratchy','scattered_light','repeating_blip','power_line','paired_dove','low_freq_burst','light_modulation']):
        loc='gdrive/My Drive/GW data/Glitches/'+i+'.wav'
        rate,data=s.read(loc)
        for c1 in range(5,9):
          for c2 in range(3,13,2):
            b, a = signal.butter(c1, .9, btype='lowpass', analog=False)
            low_passed = signal.filtfilt(b, a, data)
            y = 10e-28*signal.medfilt(low_passed,c2)
            k=np.zeros(8190)
            j=0
            for i in range(0,len(y),5):
              k[j]=y[i]
              j+=1
            pos=np.random.randint(0,noise.shape[1]-len(k))
            signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(k)]+=k
            next_val+=(noise.shape[0])
            writer = csv.writer(file)
            for i in range(noise.shape[0]):
              col=np.zeros(44)
              col[34],col[43]=1,1
              writer.writerow(col)

#------------------------------------------------------------------------------
#                             CCSNe+Transient Noise               -------------
#------------------------------------------------------------------------------
  def ccsne(noise):
    global next_val,signal_gw
    signal_gw[next_val:next_val+noise.shape[0]]=np.copy(noise)
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      val=['signal_A1B1G1_R.dat','signal_A1B2G1_R.dat','signal_A1B3G1_R.dat','signal_A1B3G2_R.dat','signal_A1B3G3_R.dat','signal_A1B3G5_R.dat','signal_A2B4G1_R.dat','signal_A3B1G1_R.dat','signal_A3B2G1_R.dat','signal_A3B2G2_R.dat','signal_A3B2G4_soft_R.dat','signal_A3B2G4_R.dat','signal_A3B3G1_R.dat','signal_A3B3G2_R.dat','signal_A3B3G3_R.dat','signal_A3B3G5_R.dat','signal_A3B4G2_R.dat','signal_A3B5G4_R.dat','signal_A4B1G1_R.dat','signal_A4B1G2_R.dat','signal_A4B2G2_R.dat','signal_A4B2G3_R.dat','signal_A4B4G4_R.dat','signal_A4B4G5_R.dat','signal_A4B5G4_R.dat','signal_A4B5G5_R.dat']
      for aak in tqdm(val):
        loc='gdrive/My Drive/GW data/CCSNe/'+aak
        x, y = np.loadtxt(loc,unpack=True, usecols=[0,1])
        for r in [10,20,30]:
          for theta in [30,60]:
            y = 1/8*np.sqrt(15/np.pi)*y/r*(np.sin(theta))**2
            new_arr=np.zeros(8190)
            j=0
            for i in range(0,len(y),5):
              new_arr[j]=y[i]
              j+=1
            pos=np.random.randint(0,noise.shape[1]-len(new_arr))
            signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(new_arr)]+=new_arr
            writer = csv.writer(file)
            for i in range(noise.shape[0]):
              col=np.zeros(44)
              if val=='signal_A2B4G1_R.dat'  or   val=='signal_A3B3G5_R.dat' or  val=='signal_A3B5G4_R.dat' :
                col[37]=1
              elif val=='signal_A3B3G1_R.dat'  or val=='signal_A3B4G2_R.dat':
                col[36]=1
              elif val=='signal_A3B3G2_R.dat' or  val=='signal_A3B3G3_R.dat' or  val=='signal_A4B2G2_R.dat' or  val=='signal_A4B2G3_R.dat' or  val=='signal_A4B4G4_R.dat' or  val=='signal_A4B4G5_R.dat' or  val=='signal_A4B5G4_R.dat' or  val=='signal_A4B5G5_R.dat':
                col[36],col[35]=1,1
              else:
                col[35]=1
              col[37+r//10],col[40+theta//30],col[43]=1,1,1
              writer.writerow(col)
            next_val=next_val+noise.shape[0]    

#------------------------------------------------------------------------------
#                             MIXED SIGNALS                       -------------
#------------------------------------------------------------------------------
  def mixed_signals_BHBNSB(noise):
    global next_val,signal_gw
    apx=['TaylorT1', 'TaylorT2', 'SpinTaylorT4', 'SpinTaylorT5','EOBNRv2' , 'SEOBNRv1', 'SEOBNRv2','IMRPhenomB']
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      for a in tqdm(range(len(apx))):
        check=np.zeros(noise.shape[1])
        k=0
        spin1x,spin1y,spin1z,spin2x,spin2y,spin2z=0,0,0,0,0,0
        if apx=='TaylorT1':
          for m1 in range(5,21,5):
            for m2 in range(5,21,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)

        elif apx=='TaylorT2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    
        elif apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [60,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      spin1x=0.5,spin1y=0.8,spin1z=0.5,
                                      spin2x=0.5,spin2y=0.8,spin2z=0.5,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
          
        elif apx=='EOBNRv2' or apx=='SEOBNRv1' or apx=='SEOBNRv2':
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
        else:
          for m1 in range(5,16,5):
            for m2 in range(5,16,5):
              for d in range(50,501,50):
                for fl in [40,80,120]:
                  if (m1+m2+d+fl) not in check:
                    check[k]=m1+m2+d+fl
                    hp,hc = get_td_waveform(approximant=apx[a],
                                      mass1=m1,mass2=m2,
                                      delta_t=1.0/4096,
                                      f_lower=fl,f_final=50, 
                                      distance=d)
                    if len(hp)<=8192:
                      for loops in range(2):####10
                        t=np.linspace(0,.3,np.random.randint(8192))
                        y2=np.zeros(len(t))          
                        r=.3
                        for j in range(3,8):
                          for i in range(len(t)):
                            aa=t[i]-0.0295-j*0.0295-(j*(j+1)/2)*r*0.0295
                            y2[i]+=1.5*10e-21*(-1)**j*(1.5*10e-21*.5/(3+j))*np.exp(-(aa**2)/(2*.006**2))*np.cos(2*np.pi*250*aa)

                          #wave=['helix2']#,'whistle','wandering_line','violin_mode','Tomte']:#,'koi_fish','scratchy','scattered_light','repeating_blip','power_line','paired_dove','low_freq_burst','light_modulation']:
                          #for wave_name in wave: 
                          #  loc='gdrive/My Drive/GW data/Glitches/'+wave_name+'.wav'
                          #  rate,data=s.read(loc)
                          #  for c2 in [5,7]:
                          #    b, a = signal.butter(7, .9, btype='lowpass', analog=False)
                          #    low_passed = signal.filtfilt(b, a, data)
                          #    z = 10e-28*signal.medfilt(low_passed,c2)
                          #    sig=np.zeros(8820)
                          #    jj=0
                          #    for ii in range(0,len(z),5):
                          #      sig[jj]=z[ii]
                          #      jj+=1

                          signal_gw[next_val:next_val+noise.shape[0]]=np.copy(noise)
                          pos=np.random.randint(0,noise.shape[1]-len(hp))
                          signal_gw[next_val:next_val+noise.shape[0] ,pos:pos+len(hp)]+=hp
                          #pos=np.random.randint(0,noise.shape[1]-len(sig))
                          #signal_gw[next_val:next_val+noise.shape[0] ,pos:pos+len(sig)]+=sig
                          pos=np.random.randint(0,noise.shape[1]-len(y2))                                  
                          signal_gw[next_val:next_val+noise.shape[0] ,pos:pos+len(y2)]+=y2
                          writer = csv.writer(file)
                          for i in range(noise.shape[0]):
                            col=np.zeros(44)
                            if apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
                              col[m1//5],col[3+m2//4],col[7+d//50],col[18],col[19],col[20],col[21],col[22],col[23],col[23+fl//40],col[27]=1,1,1,1,1,1,1,1,1,1,1
                            else:
                              col[m1//5],col[3+m2//4],col[7+d//50],col[23+fl//40],col[27]=1,1,1,1,1
                            col[29],col[29+(25//10)-24]=1,1
                            writer.writerow(col)  
                            k=k+1
                          next_val+=(noise.shape[0])
                                #for i in range(noise.shape[0]):
                                #  col=np.zeros(44)
                                #  if apx=='SpinTaylorT4' or apx=='SpinTaylorT5':
                                #    col[m1//5],col[3+m2//4],col[7+d//50],col[18],col[19],col[20],col[21],col[22],col[23],col[23+fl//40],col[27]=1,1,1,1,1,1,1,1,1,1,1
                                #  else:
                                #    col[m1//5],col[3+m2//4],col[7+d//50],col[23+fl//40],col[27]=1,1,1,1,1
                                #  col[29],col[29+(25//10)-24],col[34]=1,1,1
                                #writer.writerow(col)                         




#------------------------------------------------------------------------------
#                             MIXED SIGNALS                       -------------
#------------------------------------------------------------------------------
  def mixed_signals_CCSNe(noise):
    global next_val,signal_gw  
    with open('gdrive/My Drive/GW data/labels_mixed.csv', 'a', newline='') as file:
      val=['signal_A1B1G1_R.dat','signal_A1B2G1_R.dat','signal_A1B3G1_R.dat','signal_A1B3G2_R.dat','signal_A1B3G3_R.dat','signal_A1B3G5_R.dat','signal_A2B4G1_R.dat','signal_A3B1G1_R.dat','signal_A3B2G1_R.dat','signal_A3B2G2_R.dat','signal_A3B2G4_soft_R.dat','signal_A3B2G4_R.dat','signal_A3B3G1_R.dat','signal_A3B3G2_R.dat','signal_A3B3G3_R.dat','signal_A3B3G5_R.dat','signal_A3B4G2_R.dat','signal_A3B5G4_R.dat','signal_A4B1G1_R.dat','signal_A4B1G2_R.dat','signal_A4B2G2_R.dat','signal_A4B2G3_R.dat','signal_A4B4G4_R.dat','signal_A4B4G5_R.dat','signal_A4B5G4_R.dat','signal_A4B5G5_R.dat']
      for aak in val:
        loc='gdrive/My Drive/GW data/CCSNe/'+aak
        x, y = np.loadtxt(loc,unpack=True, usecols=[0,1])
        for r in [10,20,30]:
          for theta in [30,60]:
            y = 1/8*np.sqrt(15/np.pi)*y/r*(np.sin(theta))**2
            new_arr=np.zeros(8190)
            j=0
            for i in range(0,len(y),5):
              new_arr[j]=y[i]
              j+=1
            wave=['helix2','whistle','wandering_line','violin_mode','Tomte','koi_fish','scratchy','scattered_light','repeating_blip','power_line','paired_dove','low_freq_burst','light_modulation']
            for wave_name in wave: 
              loc='gdrive/My Drive/GW data/Glitches/'+wave_name+'.wav'
              rate,data=s.read(loc)
              for c2 in [5,7]:
                b, a = signal.butter(7, .9, btype='lowpass', analog=False)
                low_passed = signal.filtfilt(b, a, data)
                z = 10e-28*signal.medfilt(low_passed,c2)
                sig=np.zeros(8190)
                jj=0
                for ii in range(0,len(z),5):
                  sig[jj]=z[ii]
                  jj+=1
                signal_gw[next_val:next_val+noise.shape[0]]=np.copy(noise)
                pos=np.random.randint(0,noise.shape[1]-len(sig)) 
                signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(sig)]+=sig 
                pos=np.random.randint(0,noise.shape[1]-len(new_arr))
                signal_gw[next_val:next_val+noise.shape[0],pos:pos+len(new_arr)]+=new_arr                              
                writer = csv.writer(file)
                for i in range(noise.shape[0]):
                  col=np.zeros(44)
                  if val=='signal_A2B4G1_R.dat' or  val=='signal_A3B3G5_R.dat' or  val=='signal_A3B5G4_R.dat' :
                    col[37]=1
                  elif val=='signal_A3B3G1_R.dat' or val=='signal_A3B4G2_R.dat':
                    col[36]=1
                  elif val=='signal_A3B3G2_R.dat' or  val=='signal_A3B3G3_R.dat' or  val=='signal_A4B2G2_R.dat' or  val=='signal_A4B2G3_R.dat' or  val=='signal_A4B4G4_R.dat' or  val=='signal_A4B4G5_R.dat' or  val=='signal_A4B5G4_R.dat' or  val=='signal_A4B5G5_R.dat':
                    col[36],col[35]=1,1
                  else:
                    col[35]=1
                  col[34],col[37+r//10],col[40+theta//30],col[43]=1,1,1,1
                  writer.writerow(col)
                next_val+=noise.shape[0]



#----------------------------------------------------------------------------------
#                               PIPELINES                                       ---
#----------------------------------------------------------------------------------
def train_pipeline(noise):
  global next_val,signal_gw
  val=dataprep_train
  with open('gdrive/My Drive/GW data/labels_mixed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['5','10','15','20','5','10','15','20','50','100','150','200','250','300','350','400','450','500','.5','.8','.5','.5','.8','.5','60','80','120','50','UIE','CIE','250','260','270','280','Glitch','I','II','III','10','20','30','30','60','Noise'])
  print('\nSimulated GW .......')    
  val.simulated_signals(noise)
  print('\nSimulated GW training set 100%')
  print('data size :'+ str(next_val)+'\n')  
  print('\nEchoes...')
  val.echoes(noise)
  print('data size :'+ str(next_val)+'\n')
  print('\nEchoes 100%')
  #print('\nCCSNE...')  
  #val.ccsne(noise)
  #print('data size :'+ str(next_val)+'\n')
  #print('\nCCSNE 100%')
 #print('\nGlitches...')    
 #val.glitches(noise)
 #print('data size :'+ str(next_val)+'\n')  
 #print('\nGlitches 100%')  
 #print('\nMixed training BHBNSB ...' )
  val.mixed_signals_BHBNSB(noise)
  print('\nMixed set BHBNSB 100%')
  print('data size :'+ str(next_val)+'\n')
  #print('\nMixed training CCSNe...' )
  #val.mixed_signals_CCSNe(noise)
  #print('\nMixed set CCSNe 100%')
  #print('data size :'+ str(next_val)+'\n')  
  #hf = h5py.File('gdrive/My Drive/GW data/mixed.h5', 'w')
  #hf.create_dataset('mixed', data=signal_gw)
  #hf.close()

In [ ]:
#--------------------------------------------------------
#                           MAIN                    -----
#--------------------------------------------------------
#if __name__ == '__main__':
  #global next_val
  #print('\nCollecting noise.....\n data size : '+str(next_val))
hf= h5py.File('gdrive/My Drive/GW data/noise.h5', 'r')
group_key = list(hf.keys())[0]
noise= hf[group_key]
noise=np.array(noise)
print(noise.shape,type(noise))
hf.close()
noise=noise[:5,:8192]
print(noise.shape)

(50, 16384) <class 'numpy.ndarray'>
(5, 8192)


In [ ]:
print('\n\nPreparing data..... ')
train_pipeline(noise) #--------training-------------#
print('\nPreparing data........100%\n\n')



Preparing data..... 


  0%|          | 0/8 [00:00<?, ?it/s]


Simulated GW .......


  0%|          | 0/10 [00:00<?, ?it/s]


Simulated GW training set 100%
data size :3800


Echoes...


  0%|          | 0/8 [00:00<?, ?it/s]

data size :7000


Echoes 100%


100%|██████████| 8/8 [06:22<00:00, 47.77s/it]


Mixed set BHBNSB 100%
data size :45000


Preparing data........100%




In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
data=signal_gw[:next_val]
print(data.shape)
data=data.reshape((next_val,data.shape[1],1))
print(data.shape)

(45000, 8192)
(45000, 8192, 1)


In [ ]:
labels=pd.read_csv('gdrive/My Drive/GW data/labels_mixed.csv')
labels.head

<bound method NDFrame.head of          5   10   15   20  5.1  10.1  ...  10.2  20.2   30  30.1  60.1  Noise
0      0.0  1.0  0.0  0.0  1.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    1.0
1      0.0  1.0  0.0  0.0  1.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    1.0
2      0.0  1.0  0.0  0.0  1.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    1.0
3      0.0  1.0  0.0  0.0  1.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    1.0
4      0.0  1.0  0.0  0.0  1.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    1.0
...    ...  ...  ...  ...  ...   ...  ...   ...   ...  ...   ...   ...    ...
44995  0.0  0.0  0.0  1.0  0.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    0.0
44996  0.0  0.0  0.0  1.0  0.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    0.0
44997  0.0  0.0  0.0  1.0  0.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    0.0
44998  0.0  0.0  0.0  1.0  0.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    0.0
44999  0.0  0.0  0.0  1.0  0.0   0.0  ...   0.0   0.0  0.0   0.0   0.0    0.0

[45000 rows x 44 columns]>

In [ ]:
labels=labels.to_numpy(dtype=int,copy=True)

In [ ]:
labels.shape

(45000, 44)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=1, test_size=0.2)
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(36000, 8192, 1) (36000, 44) (9000, 8192, 1) (9000, 44)


In [ ]:
model = Sequential()
model.add(Conv1D(filters=4,   kernel_size=4, activation="relu", input_shape=(X_train.shape[1],1)))#1
model.add(Dropout(0.2))
model.add(Conv1D(filters=8,   kernel_size=4, activation="relu", input_shape=(X_train.shape[1],1)))#1
model.add(Dropout(0.2))
model.add(Conv1D(filters=16,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#2
model.add(Dropout(0.2))
model.add(Conv1D(filters=32,  kernel_size=4, activation='relu',padding='valid',use_bias='True'))#3
model.add(Dropout(0.3))
model.add(Conv1D(filters=64,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#4
model.add(Dropout(0.3))
model.add(Conv1D(filters=128, kernel_size=4, activation="relu",padding='valid',use_bias='True'))#5
model.add(Dropout(0.3))
model.add(Conv1D(filters=256, kernel_size=4, activation="relu",padding='valid',use_bias='True'))#6
model.add(Dropout(0.3))
#model.add(Conv1D(filters=256, kernel_size=5, activation="relu",padding='valid',use_bias='True'))#7
#model.add(Conv1D(filters=256, kernel_size=5, activation="relu",padding='valid',use_bias='True'))#8
model.add(Conv1D(filters=128, kernel_size=4, activation="relu",padding='valid',use_bias='True'))#12
model.add(Dropout(0.2))
model.add(Conv1D(filters=64,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#13
model.add(Dropout(0.2))
model.add(Conv1D(filters=32,  kernel_size=4, activation="relu",padding='valid',use_bias='True'))#14
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(44, activation='sigmoid'))

In [ ]:
sgd = SGD(lr=1e-6,decay=1e-6, momentum=0.99,nesterov=True, clipnorm=1.0)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
a=model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=100)

Train on 36000 samples, validate on 9000 samples
Epoch 1/10
36000/36000 [==============================] - 577s 16ms/step - loss: 0.6931 - accuracy: 0.9086 - val_loss: 0.6931 - val_accuracy: 0.9102
Epoch 2/10
12000/36000 [=========>....................] - ETA: 5:55 - loss: 0.6931 - accuracy: 0.9106

In [ ]:
plt.plot(a.history['accuracy'])
plt.plot(a.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()